# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # импорт библиотеки pandas и ее запись в переменной pd
from pymystem3 import Mystem # импорт функции Mystem из библиотеки pymystem3  
from collections import Counter # импорт функции Counter из библиотеки collections

In [2]:
data = pd.read_csv('/datasets/data.csv') # читаем исходную таблицу и записываем ее в переменную data
print(data.head()) # для ознакомления с таблицей напечатаем 10 первых строк из таблицы
data.info() # применяем метод .info к исходным данным, чтобы их проанализировать.
data.describe() # применяем метод .describe к исходным данным, чтобы их проанализировать.

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

При первом взгляде на таблицу можно заметить, что есть пропущенные значения с столбцах days_employed и total_income. Так же в исходной таблице можно выделить следующие типы переменных: логические - debt; количественные - children, days_employed, dob_years, total_income; остальные данные будут относится к категориальным переменным. Так как у нас пропущенные значения в количественных переменных, то будем использовать характерные значения из нашей выборки, применяя к ним методы нахождения среднего арифметического или медианы. Кроме того, явная проблема с данными в столбце days_employed: большинство из них отрицательные, за исключением тех, где income_type = пенсионер (и не смотря на то, что они положительные, они чрезмерно велики для трудового стажа). Так же тип данных для столбца days_employed (общий трудовой стаж в днях) не является целым числом, а дробным, с плавающей точкой (float64).

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
print(data.isnull().sum()) # находим количество пропущенных значений

# заполним пропущенные значения в столбце total_income средними значениями, в зависимости от типа занятости:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('mean')) 

# заполним пропущенные значения в столбце days_employed средними значениями, в зависимости от типа занятости:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))

data.info() # визуальная проверка данных чтобы убедиться, что у нас нет пропущенных значений


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы избавились от пропущенных значений в наших данных. Пропущенные значения в столбце ежемесячного дохода мы заменили на среднее арифметическое, а пропущенные значения в столбце трудовой стаж заменили на среднее, посчитанное методом медианы. Применили метод медианы так как в данных есть артефакты, которые сильно выделяются по значениям (где income_type = пенсионер). Пропуски в данных могли образоваться из-за того, что эти данные не были заполнены в заявке при оформлении кредита. Отрицательные значения трудового стажа могли образоваться из-за разности: "начало трудовой деятельности" - "дата заполнения заявки на кредит" (или "дата выдачи кредита"). Почему у пенсионеров получились такие огромные цифры трудового стажа, сложно объяснить, возможно эти значения прописвыаются автоматически, если клиент указал, что является пенсионером.

### Замена типа данных

In [4]:
data['education'] = data['education'].str.lower() # приведем все значения столбца education книжнему регистру
# переводим значения трудового стажа из вещественного типа в целочисленный тип данных
data['days_employed'] = data['days_employed'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Для данных из столбца трудового стажа изменили тип данных с вещественного на целочисленный

### Обработка дубликатов

In [5]:
print(data.duplicated().sum()) # находим количество дубликатов в данных
data = data.drop_duplicates().reset_index(drop = True) # удаляем дубликаты из данных и перезаписываем индексы
print(data.duplicated().sum()) # проверяем остались ли дубликаты

71
0


**Вывод**

Удалили дубликаты из данных, так же удалили номера индексов для дубликатов и перезаписали их по поряку.

### Лемматизация

In [6]:
m = Mystem()
lemmas = data['purpose'].apply(lambda x: ' '.join(m.lemmatize(x))) # применим лемматизацию для каждого значения из столбца purpose 
print(Counter(lemmas)) # вывод на экран числа упомянутых причин для получения кредита


Counter({'автомобиль \n': 972, 'свадьба \n': 791, 'на   проведение   свадьба \n': 768, 'сыграть   свадьба \n': 765, 'операция   с   недвижимость \n': 675, 'покупка   коммерческий   недвижимость \n': 661, 'операция   с   жилье \n': 652, 'покупка   жилье   для   сдача \n': 651, 'операция   с   коммерческий   недвижимость \n': 650, 'покупка   жилье \n': 646, 'жилье \n': 646, 'покупка   жилье   для   семья \n': 638, 'строительство   собственный   недвижимость \n': 635, 'недвижимость \n': 633, 'операция   со   свой   недвижимость \n': 627, 'строительство   жилой   недвижимость \n': 624, 'покупка   недвижимость \n': 621, 'покупка   свой   жилье \n': 620, 'строительство   недвижимость \n': 619, 'ремонт   жилье \n': 607, 'покупка   жилой   недвижимость \n': 606, 'на   покупка   свой   автомобиль \n': 505, 'заниматься   высокий   образование \n': 496, 'сделка   с   подержанный   автомобиль \n': 486, 'на   покупка   подержать   автомобиль \n': 478, 'свой   автомобиль \n': 478, 'на   покупка   ав

**Вывод**

Чаще всего за кредитом обращаются с целью приобретения автомобиля и недвижимости, реже - на оплату дополнительного образования и празднование свадьбы.

### Категоризация данных

In [7]:
# выделяем столбцы с типом занятости и наличием просрочек по кредиту в отдельную таблицу:
income_type_dict = data[['income_type', 'debt']]
# группируем новую таблицу по типу занятости и считаем суммарное кол-во просрочек, сохраняя в отдельную переменную:
income_type_dict_1 = income_type_dict.groupby('income_type')['debt'].sum()
# группируем новую таблицу по типу занятости и считаем кол-во выданных кредитов, сохраняя в отдельную переменную:
income_type_dict_2 = income_type_dict.groupby('income_type')['debt'].count()
print(income_type_dict) # вывод новой таблицы с типом занятости и наличием просрочек по кредиту
print(income_type_dict_1) # вывод сгруппированной таблицы по типу занятости и кол-вом просрочек
print(income_type_dict_2) # вывод сгруппированной таблицы по типу занятости и кол-ву выданных кредитов

      income_type  debt
0       сотрудник     0
1       сотрудник     0
2       сотрудник     0
3       сотрудник     0
4       пенсионер     0
...           ...   ...
21449   компаньон     0
21450   пенсионер     0
21451   сотрудник     1
21452   сотрудник     1
21453   сотрудник     0

[21454 rows x 2 columns]
income_type
безработный           1
в декрете             1
госслужащий          86
компаньон           376
пенсионер           216
предприниматель       0
сотрудник          1061
студент               0
Name: debt, dtype: int64
income_type
безработный            2
в декрете              1
госслужащий         1457
компаньон           5078
пенсионер           3829
предприниматель        2
сотрудник          11084
студент                1
Name: debt, dtype: int64


In [8]:
# выделяем столбцы с кол-вом детей и наличием просрочек по кредиту в отдельную таблицу:
children_dict = data[['children', 'debt']]
# группируем новую таблицу по кол-ву детей и считаем суммарное кол-во просрочек, сохраняя в отдельную переменную:
children_dict_1 = children_dict.groupby('children')['debt'].sum()
# группируем новую таблицу по кол-ву детей и считаем кол-во выданных кредитов, сохраняя в отдельную переменную:
children_dict_2 = children_dict.groupby('children')['debt'].count()
print(children_dict) # вывод новой таблицы с количеством детей и наличием просрочек по кредиту
print(children_dict_1) # вывод сгруппированной таблицы по кол-ву детей и кол-вом просрочек
print(children_dict_2) # вывод сгруппированной таблицы по кол-ву детей и кол-ву выданных кредитов

       children  debt
0             1     0
1             1     0
2             0     0
3             3     0
4             0     0
...         ...   ...
21449         1     0
21450         0     0
21451         1     1
21452         3     1
21453         2     0

[21454 rows x 2 columns]
children
-1        1
 0     1063
 1      444
 2      194
 3       27
 4        4
 5        0
 20       8
Name: debt, dtype: int64
children
-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64


In [9]:
# выделяем столбцы с семейным положением, индификатором семейного положения и наличием просрочек по кредиту в отдельную таблицу:
family_status_dict = data[['family_status', 'family_status_id', 'debt']]
# группируем новую таблицу по индификатору семейного положения и считаем суммарное кол-во просрочек, сохраняя в отдельную переменную:
family_status_dict_1 = family_status_dict.groupby('family_status_id')['debt'].sum()
# группируем новую таблицу по индификатору семейного положения и считаем кол-во выданных кредитов, сохраняя в отдельную переменную:
family_status_dict_2 = family_status_dict.groupby('family_status_id')['debt'].count()
print(family_status_dict) # вывод новой таблицы с семейным положением, индификатору семейного положения и наличием просрочек по кредиту
print(family_status_dict_1) # вывод сгруппированной таблицы по индификатору семейного положения и кол-вом просрочек
print(family_status_dict_2) # вывод сгруппированной таблицы по индификатору семейного положения и кол-ву выданных кредитов

          family_status  family_status_id  debt
0       женат / замужем                 0     0
1       женат / замужем                 0     0
2       женат / замужем                 0     0
3       женат / замужем                 0     0
4      гражданский брак                 1     0
...                 ...               ...   ...
21449  гражданский брак                 1     0
21450   женат / замужем                 0     0
21451  гражданский брак                 1     1
21452   женат / замужем                 0     1
21453   женат / замужем                 0     0

[21454 rows x 3 columns]
family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64
family_status_id
0    12339
1     4151
2      959
3     1195
4     2810
Name: debt, dtype: int64


In [10]:
# выделяем столбцы с образованием, индификатором образования и наличием просрочек по кредиту в отдельную таблицу:
education_dict = data[['education', 'education_id', 'debt']]
# группируем новую таблицу по индификатору образования и считаем суммарное кол-во просрочек, сохраняя в отдельную переменную:
education_dict_1 = education_dict.groupby('education_id')['debt'].sum()
# группируем новую таблицу по индификатору образования и считаем кол-во выданных кредитов, сохраняя в отдельную переменную:
education_dict_2 = education_dict.groupby('education_id')['debt'].count()
print(education_dict) # вывод новой таблицы с образованием, индификатором образования и наличием просрочек по кредиту
print(education_dict_1) # вывод сгруппированной таблицы по индификатору образования и кол-вом просрочек
print(education_dict_2) # вывод сгруппированной таблицы по индификатору образования и кол-ву выданных кредитов

      education  education_id  debt
0        высшее             0     0
1       среднее             1     0
2       среднее             1     0
3       среднее             1     0
4       среднее             1     0
...         ...           ...   ...
21449   среднее             1     0
21450   среднее             1     0
21451   среднее             1     1
21452   среднее             1     1
21453   среднее             1     0

[21454 rows x 3 columns]
education_id
0     278
1    1364
2      68
3      31
4       0
Name: debt, dtype: int64
education_id
0     5250
1    15172
2      744
3      282
4        6
Name: debt, dtype: int64


In [11]:
debt_depend_on_education = [] # создаем пустой список для значений просроченной задолжности в процентах
for i in range(len(education_dict_1)): # объявляем цикл по всей длине сгруппированной таблицы
    # считаем отношение количества кредитов с просроченной задолжность к общему количеству кредитов и переводим в %, умножив на 100:
    element = (education_dict_1[i] / education_dict_2[i]) * 100 
    debt_depend_on_education.append(element) # добавляем посчитанное отношение в созданный ранее пустой список
print(debt_depend_on_education) # вывод списка с посчитанными элементами

[5.295238095238095, 8.990245188505142, 9.13978494623656, 10.99290780141844, 0.0]


In [12]:
education_names = data['education'].unique() # сохраняем в переменную список с уникальными значениями из столбца education
print(education_names) # вывод на экран списка с уникальными значениями из столбца education

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [13]:
result = [] # создаем пустой список для создания списка списков
for i in range(len(education_dict_1)): # объявляем цикл по всей длине списка education_dict_1
    # создаем список для шага цикла и записываем туда элементы через запятую
    item = [education_names[i], education_dict_1[i], education_dict_2[i], debt_depend_on_education[i]] 
    result.append(item) # добавляем список, созданный на каждом шаге цикла в пустой список, созданный вне цикла
print(result) # вывод на экран списока списков

[['высшее', 278, 5250, 5.295238095238095], ['среднее', 1364, 15172, 8.990245188505142], ['неоконченное высшее', 68, 744, 9.13978494623656], ['начальное', 31, 282, 10.99290780141844], ['ученая степень', 0, 6, 0.0]]


In [14]:
# создаем список с названиями для столбцов
columns_for_final_result = ['education', 'part_of_credits_with_delayed', 'total_qty_of_credits', 'percentage_of_arrears']
# создаем таблицу методом DataFrame, где столбцы принимают значения из списка columns_for_final_result,
# а данные берутся из списка списков result
final_result_depend_on_education = pd.DataFrame(data=result , columns=columns_for_final_result)
# вывод итоговой таблицы, где наглядно видна зависимость между уровенм образования и возвратом кредита в срок
print(final_result_depend_on_education)

             education  part_of_credits_with_delayed  total_qty_of_credits  \
0               высшее                           278                  5250   
1              среднее                          1364                 15172   
2  неоконченное высшее                            68                   744   
3            начальное                            31                   282   
4       ученая степень                             0                     6   

   percentage_of_arrears  
0               5.295238  
1               8.990245  
2               9.139785  
3              10.992908  
4               0.000000  


**Вывод**

При категоризации данных я ориентировался на те факторы, которые часто повторяются в данных и могут на возврат кредита в срок. Такими категориями стали: тип занятости, наличие детей, уровень образования и семейное положение. Все эти данные сводились в таблицу по очереди с данными по задержке в выплате кредитов. Например, видна зависимость между уровнем образования и возваратом кредита в срок: чем ниже уровень образования, тем выше вероятность того, что клиент допустит очередной платёж по кредиту.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [15]:
# напишем функцию для определения к какой категории относится клиент в зависимости от количества детей:

def children_group(childred):
    if childred <= 0:
        return 'нет детей'
    if childred == 1:
        return 'один ребёнок'
    if childred == 2:
        return 'двое детей'
    else:
        return 'многодетные'

In [16]:
# добавляем новый столбец в таблицу с данными,
# где с помощью нашей функции укажем к какой категории по количеству детей относится клиент:    
data['children_group'] = data['children'].apply(children_group)

# группируем таблицу по children_group, считаем суммарное кол-во просрочек и и считаем кол-во выданных кредитов:
data_grouped_by_children_group = data.groupby('children_group')['debt'].agg(['sum', 'count'])

# посчитаем в % выражении отношение кредитов с просроченной задолжностью к общему количеству кредитов:
data_grouped_by_children_group['percentage_of_arrears'] = data_grouped_by_children_group['sum'] / data_grouped_by_children_group['count'] * 100

print(data_grouped_by_children_group) # вывод результатов

                 sum  count  percentage_of_arrears
children_group                                    
двое детей       194   2052               9.454191
многодетные       39    456               8.552632
нет детей       1064  14138               7.525817
один ребёнок     444   4808               9.234609


**Вывод**

Наличие детей в семье незначительно увеличивает (на 1-2%) шанс того, что кредит не будет возвращён в срок. В текущих данных есть исключение в виде нескольких семей, где есть 5ро детей, и не было задержек в выплате кредита. Этим исключением можно пренебречь в нашем исследовании, так как таких семей значительно меньше, чем других семей с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
# считаем отношение количества кредитов с просроченной задолжность к общему количеству кредитов и переводим в %, умножив на 100:
debt_depend_on_family_status = family_status_dict_1 / family_status_dict_2 * 100

In [18]:
family_status_names = data['family_status'].unique() # сохраняем в переменную список с уникальными значениями из столбца family_status
print(family_status_names) # вывод на экран списка с уникальными значениями из столбца family_status

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [19]:
result = [] # создаем пустой список для создания списка списков
for i in range(len(family_status_dict_1)): # объявляем цикл по всей длине списка family_status_dict_1
    # создаем список для шага цикла и записываем туда элементы через запятую
    item = [family_status_names[i], family_status_dict_1[i], family_status_dict_2[i], debt_depend_on_family_status[i]] 
    result.append(item) # добавляем список, созданный на каждом шаге цикла в пустой список, созданный вне цикла
print(result) # вывод на экран списока списков

[['женат / замужем', 931, 12339, 7.5451819434313965], ['гражданский брак', 388, 4151, 9.347145266200915], ['вдовец / вдова', 63, 959, 6.569343065693431], ['в разводе', 85, 1195, 7.112970711297072], ['Не женат / не замужем', 274, 2810, 9.750889679715302]]


In [20]:
# создаем список с названиями для столбцов
columns_for_final_result = ['family_status', 'part_of_credits_with_delayed', 'total_qty_of_credits', 'percentage_of_arrears']
# создаем таблицу методом DataFrame, где столбцы принимают значения из списка columns_for_final_result,
# а данные берутся из списка списков result
final_result_depend_on_education = pd.DataFrame(data=result , columns=columns_for_final_result)
# вывод итоговой таблицы, где наглядно видно, что нет четкой зависимости между семейным положением и возвратом кредита в срок
print(final_result_depend_on_education)

           family_status  part_of_credits_with_delayed  total_qty_of_credits  \
0        женат / замужем                           931                 12339   
1       гражданский брак                           388                  4151   
2         вдовец / вдова                            63                   959   
3              в разводе                            85                  1195   
4  Не женат / не замужем                           274                  2810   

   percentage_of_arrears  
0               7.545182  
1               9.347145  
2               6.569343  
3               7.112971  
4               9.750890  


**Вывод**

Хуже всех кредит платят те, кто не состоят в браке или живут в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
# разобьем столбец total_income на 4 интервала, в каждом из которых будет приблизительно одинаковые количество клиентов:
data['salary_gruop_1'] = pd.qcut(data['total_income'], 4)

#группируем таблицу по интервалам и в каждом из них считаем количетсво
#кредитов с просроченной задолжностью и общее количество выданных кредитов:
data_grouped_by_salary_3 = data.groupby('salary_gruop_1')['debt'].agg(['sum', 'count']) 

# посчитаем в % выражении отношение кредитов с просроченной задолжностью к общему количеству кредитов
data_grouped_by_salary_3['percentage_of_arrears'] = data_grouped_by_salary_3['sum'] / data_grouped_by_salary_3['count'] * 100

print(data_grouped_by_salary_3) # вывод результатов

                           sum  count  percentage_of_arrears
salary_gruop_1                                              
(20667.263, 107623.857]    427   5364               7.960477
(107623.857, 151887.538]   475   5363               8.856983
(151887.538, 202417.461]   494   5818               8.490890
(202417.461, 2265604.029]  345   4909               7.027908


**Вывод**

Размер дохода не влияет на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
m = Mystem() 
new_list = [] # создание пустого списка
for i in range(len(data)): # объявляем цикл по всей длине нашей таблицы с дынными 
    lemmas = ' '.join(m.lemmatize(data['purpose'][i])) # лемматизируем каждое значение в столбце purpose
    new_list.append(str(lemmas)) # добавляем в список лемму в формате строки
data['lemmas_of_purpose'] = new_list # добавим новый столбец с лемматизированными значениями цели кредита 

In [23]:
# напишем функцию для определения причины взятие кредита в одно слово:
# функция должна возвращать значения жилье, недвижимость, свадьба, автомобиль или образование

def purpose_of_credit(lemmas_of_purpose):
    if 'жилье' in lemmas_of_purpose:
        return 'жилье'
    if 'автомобиль' in lemmas_of_purpose:
        return 'автомобиль'
    if 'недвижимость' in lemmas_of_purpose:
        return 'недвижимость'
    if 'свадьба' in lemmas_of_purpose:
        return 'свадьба'
    if 'образование' in lemmas_of_purpose:
        return 'образование'

In [24]:
data['purpose_of_credit'] = data['lemmas_of_purpose'].apply(purpose_of_credit) # применяем нашу функцию к столбцу lemmas_of_purpose
# группируем таблицу по цели кредита и считаем суммарное кол-во просрочек, сохраняя в отдельную переменную:
data_grouped_by_purpose_of_credit_1 = data.groupby('purpose_of_credit')['debt'].sum()
# группируем таблицу по цели кредита и считаем кол-во выданных кредитов, сохраняя в отдельную переменную:
data_grouped_by_purpose_of_credit_2 = data.groupby('purpose_of_credit')['debt'].count()
# считаем отношение количества кредитов с просроченной задолжность к общему количеству кредитов и переводим в %, умножив на 100:
debt_depend_on_purpose_of_credit = data_grouped_by_purpose_of_credit_1 / data_grouped_by_purpose_of_credit_2 * 100
print(debt_depend_on_purpose_of_credit)

purpose_of_credit
автомобиль      9.359034
жилье           6.905830
недвижимость    7.463392
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64


**Вывод**

Клиенты, которые взяли кредит для улучшения жилищных условий платят немного лучше тех клиентов, которые взяли кредит на другие цели.

## Шаг 4. Общий вывод

Наш анализ показал, что клиент, которые состоят в браке чаще возвращают кредит в срок. С другой стороны, если в семье есть дети, то это увеличивает шанс того, что кредит не смогут вернуть в срок, так как клиенты без детей чаще возвращают кредит в срок. Возможно, это связано с тем, что если у клиента есть дети, то у него чаще возникают непредвиденные расходы и не всегда удается вернуть кредит в срок. Так же в ходе анализа удалось выяснить, что кредит, взятый с целью улучшения жилищных условий выплачивается лучше, нежели кредит взятый на другие цели. Размер дохода не влияет на возврат кредита в срок, а оразование клиента, наоборот, влияет: образованные люди чаще возвращают кредит в срок.